In [1]:
## imports
import os 

# import numpy as np
# import cv2 as cv
import json
from datetime import datetime  

# %matplotlib nbagg
%matplotlib inline
import matplotlib.pyplot as plt

# from skimage import data, img_as_float
# from skimage.segmentation import (morphological_chan_vese, checkerboard_level_set)

%reload_ext autoreload
%autoreload 2
%aimport OCT_lib

In [2]:
### Files management
## You may provide the `path` to the folder with all the binary files
# initial_path = r"C:\Users\user\Google Drive\01 - OCT\Measurements\20190708_CscanThumbRel"
# initial_path = r"C:\Users\lucab\Desktop\20190718_PDMS_Sample20180219_2"
initial_path = r"/Users/lucab/Google Drive/01 - OCT/Measurements/20190410 - step PDMS 2"

if 'initial_path' in locals():
    files, _ = OCT_lib.get_files(directory = initial_path) 
else:
    # if `initial_path` is not provided, a selector windows will appear (sometimes under the browser) 
    files, initial_path = OCT_lib.get_files()
    
folder_name = os.path.basename(os.path.normpath(initial_path))

## rolling the list of files 
## e.g.: if when acquiring a C-scan, the first slice of the intended output C-scan was saved by 
## LabView in position 74, use N_roll = 74
## (this part will be worked out on the LabView side)
N_roll = 0 #230
if N_roll:
    print(f"WARNING: the ordering of the files has been changed. \nThe first file analyzed is now B-scan #{N_roll}") 
    OCT_lib.roll_list(files, N_roll)

Found 12 files 




In [3]:
print("Analyzing only the first three files")
# files = files[:3]

Analyzing only the first three files


In [3]:
### Analysis parameters

# will show a bit more information during the execution of the code
debug_here = False 

# default parameters are stored in OCT_lib.py
default_analysis_settings = False
# if enabled, prompts the selection of a txt files with the analysis parameters in JSON format
read_parameters_from_file = False

if default_analysis_settings:
    p = OCT_lib.default_settings
    print(f"The parameters used for this analysis are the default ones")
elif read_parameters_from_file:
    param_textfile = f"{os.path.join(initial_path,'processed')}\\parameters.txt"
#     param_textfile = OCT_lib.get_file(initdir = initial_path, ext='txt')
    print(f"The parameters used for this analysis are being taken from: \n{param_textfile}") 
    with open(param_textfile, 'r') as file:
        p = json.loads(file.read()) 
else:
    print(f"The parameters used for this analysis are specified in the Jupyter Notebook") 
    ## dictionary will all the parameters for the analysis
    p = dict(
        ## if crop['switch'] is True, the image gets cropped at the specified indices 
        crop = dict(
            # turn the crop switch ON or OFF
            switch = False,
            ## cropping indices
            left = None,
            right = None,
            top = None,
            bottom = None,
        ),
        ## if resize['switch'] is True, the image gets resized with the specified factors 
        resize = dict(
            switch = False,
            width_factor = 1,
            height_factor = 1,
        ),
        ## filter_settings contains all the parameters of each filtering mode 
        filtering = dict(
                switch = True, # Turns filtering ON or OFF 
                mode = 'bilateral', # filtering mode: set to `NLM` or `bilateral`
                bilateral = dict(
                    d = 15,
                    sigma_color = 90,
                    sigma_space = 80,
                ),
                NLM = dict(
                    sigma = 6, #parameter for NLM denoising
                ),
            ),
        ## Parameters of segmentation
        segmentation = dict(
            switch = True, 
            mode = "otsu", # or  "chan_vese"
            ignore_top_px = 150,
            N_iter = 20, # used in ACWE segm. (see docs of `morphological snakes`)
        ),
        ## Parameters of Axsun OCT
        OCT = dict(
            axial_res = 5.75, # Vertical Resolution of OCT, in micron per pixel
            aperture_size = 4, # size of the suction aperture
            bypassmode = 8, # this translates to how many bytes per px  (defaul 1b/px): See Axsun manual
        ),           
        save_png = False, # saves B-scans in .png format in folder \processed
        make_res = True, # stores all processing results in a dictionary
        save_zip = True, # saves that dictionary in a pickled .bz2 
        save_settings = False, # saves analysis settings in a JSON file
    )

print(OCT_lib.print_about_settings(p))
      
if p['save_zip']:         
     assert p['make_res'], \
        "Zip file contents are based on the stored results, please set `True` the p['make_res'] switch"  

The parameters used for this analysis are specified in the Jupyter Notebook
Following settings are non-default:

p[save_png]=False DIFFERENT FROM Default_Settings[save_png](=True)
p[save_settings]=False DIFFERENT FROM Default_Settings[save_settings](=True)

Analysis settings: 
    Cropping is OFF
    Resizing is OFF
   Filtering is ON
Segmentation is ON
 .png making is OFF
    Savefile is ON


In [7]:
# initialization of the dictionary that will store the processed results
if p['make_res']:
    res = dict.fromkeys(files)

# creates folder `processed` in which to save B-scans, parameters file and/or zip file
if (p['save_png'] or p['save_zip']):
    if not os.path.exists(os.path.join(initial_path, 'processed')):
        os.makedirs(os.path.join(initial_path, 'processed'))

# ==== START cycling over the files ====
for i, file in enumerate(files[:]):
    print(f'File "{file}" - {i+1}/{len(files)}')
    path2file = os.path.join(initial_path, file)
    print(path2file)
    # load file
    b = OCT_lib.bscan(path=path2file, debug=debug_here)

    # crop
    if p['crop']['switch']:
        b.crop(top=p['crop']['top'],
               bottom=p['crop']['bottom'],
               left=p['crop']['left'],
               right=p['crop']['right'])

    ## halves the size - used for 3D volumetric visualization:
    if p['resize']['switch']:
        b.resize_raw(w=b.width * p['resize']['width_factor'],
                     h=b.ascanlength * p['resize']['height_factor'])

    ## filters the image if required
    if p['filtering']['mode']:
        # The next line is a bit complicated. Apply_filter takes in some keyword arguments
        # that are packed into the settings dictionary p['filtering'][filtermode]
        b.apply_filter(mode=p['filtering']['mode'],
                       **p['filtering'][p['filtering']['mode']])

    ## Surface identification, returns a 1D array with the height of the surface
    if p['segmentation']['switch']:
        profile = b.get_profile(mode=p['segmentation']['mode'],
                                exclude_top_px=p['segmentation']['ignore_top_px'])

    ## To obtain the 2D mask (called levelset), use:
#         levelset = b.get_level_set(mode='otsu')

    if p['make_res']:
        res[file] = {}
        res[file]['image'] = b.raw
        # relaxed or suction?
        if 'suc' in file:
            res[file]['pressure'] = 'suction'
        elif 'rel' in file:
            res[file]['pressure'] = 'relaxed'

        if p['segmentation']['switch']:
            if 'profile' in locals():
                # 1D array of skin height along the B-scan
                res[file]['profile'] = profile
            if 'levelset' in locals():
                # B&W image returned by segmentation
                res[file]['levelset'] = levelset

    # saving png image
    if p['save_png']:
        png_name = file
        profile = profile if p['segmentation']['switch'] else None
        b.save_png(
            name=png_name,
            profile=profile,
        )
# === END OF FILEs CYCLING ===

# Reads the time at which the analysis has been executed
analysis_time = datetime.now().strftime("%Y%m%d_%H-%M")

# also save analysis parameters in a .txt file (for further use)
if p['save_settings']:
    settings_filename = f"{analysis_time}_parameters.txt"
    settings_pathfile = os.path.join(
        initial_path, 'processed', settings_filename)
    print(
        f"The parameters used for this analysis have been written at: \n{settings_pathfile}")
    with open(settings_pathfile, 'w') as file: 
        # use `json.loads` to do the reverse
        file.write(json.dumps(p, indent=3))

# save all processed data in a zip file
if p['save_zip']:
    experiment = {}
    experiment['results'] = res
    experiment['params'] = p
    experiment['name'] = folder_name 
    import bz2  # to zip the pickle file:
    import pickle
#     pickle_name = (os.path.join(initial_path,'processed'))+'\\'+folder_name'+'_processed.bz2'
#     pickle_name = f"{os.path.join(initial_path,'processed')}\\{folder_name}_{analysis_time}_processed.bz2"
    pickle_name = os.path.join(initial_path,'processed',f"{folder_name}_{analysis_time}_processed.bz2")   
    print(f"Saving file at path: {pickle_name}")
    sfile = bz2.BZ2File(pickle_name, 'w')
    pickle.dump(experiment, sfile)
    sfile.close()

File "2boundary_rel" - 1/12
/Users/lucab/Google Drive/01 - OCT/Measurements/20190410 - step PDMS 2\2boundary_rel
Bypass not specified, will read the binary in chunks of 1 byte
"2boundary_rel" loaded.
 - 1024x636px, bypass: 8

File "2boundary_suc" - 2/12
/Users/lucab/Google Drive/01 - OCT/Measurements/20190410 - step PDMS 2\2boundary_suc
Bypass not specified, will read the binary in chunks of 1 byte
"2boundary_suc" loaded.
 - 1024x636px, bypass: 8

File "2thick_rel" - 3/12
/Users/lucab/Google Drive/01 - OCT/Measurements/20190410 - step PDMS 2\2thick_rel
Bypass not specified, will read the binary in chunks of 1 byte
"2thick_rel" loaded.
 - 1024x636px, bypass: 8

File "2thick_suc" - 4/12
/Users/lucab/Google Drive/01 - OCT/Measurements/20190410 - step PDMS 2\2thick_suc
Bypass not specified, will read the binary in chunks of 1 byte
"2thick_suc" loaded.
 - 1024x636px, bypass: 8

File "2thin_rel" - 5/12
/Users/lucab/Google Drive/01 - OCT/Measurements/20190410 - step PDMS 2\2thin_rel
Bypass no

In [ ]:
data[files[1]].keys()

In [8]:
experiment.keys()

dict_keys(['results', 'params', 'name'])